In [1]:
import pandas as pd
import os
import numpy as np
import os
import random

In [2]:
kmer_half = 5

In [3]:
fasta_paths_df = pd.read_csv("./231121.MtbSetV3.151CI.HybridAndSRAsm.FAPATHs.V1.tsv", sep="\t")
fasta_paths_df.head()

,SampleID,Dataset_Tag,Genome_ASM_PATH,ShortRead_Genome_ASM_PATH
0,N0072,ChinerOms_2019,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...
1,N0153,ChinerOms_2019,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...
2,TB3113,TB_Portals_24CI_R1,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...
3,TB1236,TB_Portals_24CI_R1,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...
4,TB2659,TB_Portals_24CI_R1,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...,/n/data1/hms/dbmi/farhat/mm774/Projects/231121...


In [4]:
l1_isolate = "N0072"
l2_isolate = "M0003941_3"
l3_isolate = "N0004"
l4_isolate = "M0017522_5"

In [5]:
l1_fa_path = fasta_paths_df.loc[fasta_paths_df.SampleID == l1_isolate]["Genome_ASM_PATH"].values[0]
l2_fa_path = fasta_paths_df.loc[fasta_paths_df.SampleID == l2_isolate]["Genome_ASM_PATH"].values[0]
l3_fa_path = fasta_paths_df.loc[fasta_paths_df.SampleID == l3_isolate]["Genome_ASM_PATH"].values[0]
l4_fa_path = fasta_paths_df.loc[fasta_paths_df.SampleID == l4_isolate]["Genome_ASM_PATH"].values[0]

In [6]:
h37rv_fasta = "/n/data1/hms/dbmi/farhat/mtb_data/h37rv/h37rv.fna"
with open(h37rv_fasta, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    h37rv_seq = line.strip()
    while line:
        line = in_f.readline()
        h37rv_seq += line.strip()

In [7]:
h37rv_summary = pd.read_csv("/n/data1/hms/dbmi/farhat/mtb_data/mycobrowser/h37rv_mycobrowser_genome_summary.csv").set_index("gene_name")
h37rv_summary.head(2)

,gene_id,chrom_start,chrom_stop,strand,gene_category_vargas2021,feature,function,product,functional_category,PFAM,gene_ontology,dr_mutations
gene_name,,,,,,,,,,,,
dnaA,Rv0001,1,1524,+,Essential,CDS,Plays an important role in the initiation and ...,Chromosomal replication initiator protein DnaA,information pathways,P49993,"GO:0003688,GO:0006270,GO:0005737,GO:0017111,GO...",NaN
dnaN,Rv0002,2052,3260,+,Non-Essential,CDS,"DNA polymerase III is a complex, multichain en...",DNA polymerase III (beta chain) DnaN (DNA nucl...,information pathways,Q50790,"GO:0003677,GO:0006260,GO:0003887,GO:0005737,GO...",NaN


# DR mutations

In [53]:
dr_mutations = pd.read_csv("/home/sm624/data/antib_resistance/WHO_drug_SNP_mapping.csv")
print(dr_mutations.shape)
dr_mutations.head()

(513, 7)


,gene_name,gene_locus,variant,codon_number,genome_index,nt_change,drug
0,gyrB,Rv0005,gyrB_D461N,461.0,6620,A,LEVO
1,gyrB,Rv0005,gyrB_D461N,461.0,6620,A,MOXI
2,gyrB,Rv0005,gyrB_N499D,499.0,6734,G,LEVO
3,gyrB,Rv0005,gyrB_N499D,499.0,6734,G,MOXI
4,gyrB,Rv0005,gyrB_E501V,501.0,6741,T,LEVO


In [15]:
DR_subst_ix_base_list = []

for i in dr_mutations.index:
    
    pos_base = (dr_mutations.loc[i, "genome_index"]-1, dr_mutations.loc[i, "nt_change"])
    
    if pos_base not in DR_subst_ix_base_list:
        DR_subst_ix_base_list.append(pos_base)
        
len(DR_subst_ix_base_list)

409

In [49]:
random.shuffle(DR_subst_ix_base_list)

to_mutate_ix_base_list = []
mutated_ix = []

mutation_ix = 0

while len(to_mutate_ix_base_list) < 20:

    DR_mut_ix, DR_mut = DR_subst_ix_base_list[mutation_ix][0], DR_subst_ix_base_list[mutation_ix][1]

    if not ((DR_mut_ix in mutated_ix) or (DR_mut_ix-1 in mutated_ix) or (DR_mut_ix+1 in mutated_ix)):

        to_mutate_ix_base_list.append((DR_mut_ix, DR_mut))
        mutated_ix.append(DR_mut_ix)
        
    mutation_ix += 1
    
to_mutate_ix_base_list

[(2288777, 'G'),
 (2289071, 'G'),
 (2289049, 'G'),
 (2288961, 'C'),
 (2289096, 'A'),
 (2288919, 'A'),
 (7569, 'T'),
 (2288816, 'T'),
 (4247468, 'G'),
 (4407795, 'A'),
 (761138, 'G'),
 (2288838, 'C'),
 (4327266, 'A'),
 (1473245, 'G'),
 (7581, 'C'),
 (2288972, 'C'),
 (2288886, 'C'),
 (2289104, 'T'),
 (761136, 'T'),
 (2289102, 'G')]

In [50]:
DR_mut_kmers_df = pd.DataFrame(columns=["h37rv_ix", "mut_base", "kmer"])
df_i = 0

for ix, base in to_mutate_ix_base_list:
    
    h37rv_ref_base = h37rv_seq[ix]
    
    h37rv_kmer = h37rv_seq[ix-kmer_half:ix+kmer_half+1]
    
    new_kmer_half = kmer_half
    while h37rv_seq.count(h37rv_kmer) > 1:
        new_kmer_half += 1
        h37rv_kmer = h37rv_seq[ix-new_kmer_half:ix+new_kmer_half+1]
    
    DR_mut_kmers_df.loc[df_i] = [ix, base, h37rv_kmer]
    df_i += 1
    
DR_mut_kmers_df

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT
5,2288919,A,GACTCCTTCGA
6,7569,T,GCGACGCGTCGAT
7,2288816,T,TCGGCCGTCTGGC
8,4247468,G,CAACTATTTCC
9,4407795,A,CGCCCGTGACACCGC


In [51]:
DR_mut_kmers_df.to_csv("./DR_mut_h37rv_kmers.csv", index=False)

## Check L1 kmer matches

In [7]:
DR_mut_kmers_df = pd.read_csv("./DR_mut_h37rv_kmers.csv")
DR_mut_kmers_df.head()

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT


In [8]:
with open(l1_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L1_seq = line.strip()
    while line:
        line = in_f.readline()
        L1_seq += line.strip()

In [9]:
len(L1_seq), len(h37rv_seq)

(4421404, 4411532)

In [10]:
for i in DR_mut_kmers_df.index:
    
    kmer = DR_mut_kmers_df.loc[i, "kmer"]
    
    if L1_seq.count(kmer) != 1:
        print(DR_mut_kmers_df.loc[i])

h37rv_ix             7581
mut_base                C
kmer        TCTACGACAGCCT
Name: 14, dtype: object


## Check L2 kmer matches

In [11]:
DR_mut_kmers_df = pd.read_csv("./DR_mut_h37rv_kmers.csv")
DR_mut_kmers_df.head()

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT


In [12]:
with open(l2_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L2_seq = line.strip()
    while line:
        line = in_f.readline()
        L2_seq += line.strip()

In [13]:
len(L2_seq), len(h37rv_seq)

(4421169, 4411532)

In [14]:
for i in DR_mut_kmers_df.index:
    
    kmer = DR_mut_kmers_df.loc[i, "kmer"]
    
    if L2_seq.count(kmer) != 1:
        print(DR_mut_kmers_df.loc[i])

h37rv_ix             7581
mut_base                C
kmer        TCTACGACAGCCT
Name: 14, dtype: object


## Check L3 kmer matches

In [15]:
DR_mut_kmers_df = pd.read_csv("./DR_mut_h37rv_kmers.csv")
DR_mut_kmers_df.head()

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT


In [16]:
with open(l3_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L3_seq = line.strip()
    while line:
        line = in_f.readline()
        L3_seq += line.strip()

In [17]:
len(L3_seq), len(h37rv_seq)

(4423516, 4411532)

In [18]:
for i in DR_mut_kmers_df.index:
    
    kmer = DR_mut_kmers_df.loc[i, "kmer"]
    
    if L3_seq.count(kmer) != 1:
        print(DR_mut_kmers_df.loc[i])

h37rv_ix        2289049
mut_base              G
kmer        GAGGAATAGTC
Name: 2, dtype: object
h37rv_ix             7581
mut_base                C
kmer        TCTACGACAGCCT
Name: 14, dtype: object


## Check L4 kmer matches

In [8]:
DR_mut_kmers_df = pd.read_csv("./DR_mut_h37rv_kmers.csv")
DR_mut_kmers_df.head()

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT


In [9]:
with open(l4_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L4_seq = line.strip()
    while line:
        line = in_f.readline()
        L4_seq += line.strip()

In [10]:
len(L4_seq), len(h37rv_seq)

(4416715, 4411532)

In [11]:
for i in DR_mut_kmers_df.index:
    
    kmer = DR_mut_kmers_df.loc[i, "kmer"]
    
    if L4_seq.count(kmer) != 1:
        print(DR_mut_kmers_df.loc[i])

h37rv_ix             7581
mut_base                C
kmer        TCTACGACAGCCT
Name: 14, dtype: object


## BLAST analysis of problematic positions

In [56]:
dr_mutations[dr_mutations.genome_index == 7582]

,gene_name,gene_locus,variant,codon_number,genome_index,nt_change,drug
24,gyrA,Rv0006,gyrA_D94G,94.0,7582,G,LEVO
25,gyrA,Rv0006,gyrA_D94G,94.0,7582,G,MOXI
26,gyrA,Rv0006,gyrA_D94A,94.0,7582,C,LEVO
27,gyrA,Rv0006,gyrA_D94A,94.0,7582,C,MOXI


In [102]:
dr_mutations[dr_mutations.genome_index == 2289050]

,gene_name,gene_locus,variant,codon_number,genome_index,nt_change,drug
306,pncA,Rv2043c,pncA_Y64!,64.0,2289050,G,PZA


### Check nearby lineage markers

In [59]:
lineage_snps = pd.read_csv("/home/sm624/projects/calculate_F2/Coll2014_LinSpeSNPs_final.csv")
lineage_snps

,lineage,position,gene_coordinate,allele_change,codon_number,codon_change,amino_acid_change,locus_Id,gene_name,mutation_type,essential,diagnostic_snp
0,lineage1,6112,873,G/C,291,ATG/ATC,M/I,Rv0005,gyrB,nonsyn,essential,no
1,lineage1,8452,1151,C/T,384,GCA/GTA,A/V,Rv0006,gyrA,nonsyn,nonessential,no
2,lineage1,13298,261,G/C,87,ATC/ATG,I/M,Rv0010c,0,nonsyn,nonessential,no
3,lineage1,22961,221,T/G,74,GAT/GCT,D/A,Rv0018c,pstP,nonsyn,nonessential,no
4,lineage1,26347,535,C/G,179,GAC/CAC,D/H,Rv0021c,0,nonsyn,nonessential,no
...,...,...,...,...,...,...,...,...,...,...,...,...
6910,lineageBOV_AFRI,4292095,1080,C/A,360,GCG/GCT,A/A,Rv3824c,papA1,syn,nonessential,no
6911,lineageBOV_AFRI,4330238,1268,C/T,423,CGT/CAT,R/H,Rv3858c,gltD,nonsyn,essential,no
6912,lineageBOV_AFRI,4386257,52,G/C,18,CCG/GCG,P/A,Rv3900c,0,nonsyn,nonessential,no
6913,lineageBOV_AFRI,4393838,390,C/T,130,CTC/CTT,L/L,Rv3908,mutT4,syn,nonessential,no


In [73]:
lineage_snps[lineage_snps.gene_name == "gyrA"]

,lineage,position,gene_coordinate,allele_change,codon_number,codon_change,amino_acid_change,locus_Id,gene_name,mutation_type,essential,diagnostic_snp
1,lineage1,8452,1151,C/T,384,GCA/GTA,A/V,Rv0006,gyrA,nonsyn,nonessential,no
931,lineage1.2.1,9260,1959,G/C,653,CTG/CTC,L/L,Rv0006,gyrA,syn,nonessential,no
2180,lineage3.1.2.2,9611,2310,C/T,770,GAC/GAT,D/D,Rv0006,gyrA,syn,nonessential,no
3650,lineage4.3.3,8040,739,G/A,247,GGC/AGC,G/S,Rv0006,gyrA,nonsyn,nonessential,no
4263,lineage4.5,7892,591,G/A,197,CTG/CTA,L/L,Rv0006,gyrA,syn,nonessential,no
4422,lineage4.6.1,7539,238,A/G,80,ACC/GCC,T/A,Rv0006,gyrA,nonsyn,nonessential,no
5211,lineage5,9566,2265,C/T,755,TAC/TAT,Y/Y,Rv0006,gyrA,syn,nonessential,no
5804,lineage7,8876,1575,C/T,525,TAC/TAT,Y/Y,Rv0006,gyrA,syn,nonessential,no


In [72]:
lineage_snps[lineage_snps.gene_name == "pncA"]

,lineage,position,gene_coordinate,allele_change,codon_number,codon_change,amino_acid_change,locus_Id,gene_name,mutation_type,essential,diagnostic_snp


### Create gene fasta files

In [31]:
genes = ["gyrA", "pncA"]

In [32]:
h37rv_summary = pd.read_csv("/n/data1/hms/dbmi/farhat/mtb_data/mycobrowser/h37rv_mycobrowser_genome_summary.csv").set_index("gene_name")
h37rv_summary.head(2)

,gene_id,chrom_start,chrom_stop,strand,gene_category_vargas2021,feature,function,product,functional_category,PFAM,gene_ontology,dr_mutations
gene_name,,,,,,,,,,,,
dnaA,Rv0001,1,1524,+,Essential,CDS,Plays an important role in the initiation and ...,Chromosomal replication initiator protein DnaA,information pathways,P49993,"GO:0003688,GO:0006270,GO:0005737,GO:0017111,GO...",NaN
dnaN,Rv0002,2052,3260,+,Non-Essential,CDS,"DNA polymerase III is a complex, multichain en...",DNA polymerase III (beta chain) DnaN (DNA nucl...,information pathways,Q50790,"GO:0003677,GO:0006260,GO:0003887,GO:0005737,GO...",NaN


In [81]:
# chrom_start and chrom_stop are inclusive

for gene in genes:
    
    chrom_start, chrom_stop = h37rv_summary.loc[gene, "chrom_start"], h37rv_summary.loc[gene, "chrom_stop"]
    
    gene_seq = h37rv_seq[chrom_start-1:chrom_stop]
    
    with open("./blast/{}.fa".format(gene), "w") as out_f:
        out_f.write(">{}\n".format(gene))
        n_chunks = len(gene_seq)//80
        for i in range(n_chunks):
            out_f.write(gene_seq[i*80:i*80+80] + "\n")
        out_f.write(gene_seq[(i+1)*80:] + "\n")
    

```
makeblastdb -in gyrA.fa -dbtype nucl

makeblastdb -in pncA.fa -dbtype nucl
```

```
## gyrA
### lineage 1
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0072.LR.Asm.fasta -db gyrA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage1.gyrA.blastn.txt

### lineage 2
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/Farhat_Peru_2019/M0003941_3.LR.Asm.fasta -db gyrA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage2.gyrA.blastn.txt

### lineage 3
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0004.LR.Asm.fasta -db gyrA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage3.gyrA.blastn.txt

### lineage 4
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/Farhat_Peru_2019/M0017522_5.LR.Asm.fasta -db gyrA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage4.gyrA.blastn.txt
```

```
## pncA
### lineage 1
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0072.LR.Asm.fasta -db pncA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage1.pncA.blastn.txt

### lineage 2
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/Farhat_Peru_2019/M0003941_3.LR.Asm.fasta -db pncA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage2.pncA.blastn.txt

### lineage 3
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0004.LR.Asm.fasta -db pncA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage3.pncA.blastn.txt

### lineage 4
blastn -query /n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/Farhat_Peru_2019/M0017522_5.LR.Asm.fasta -db pncA.fa -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq nident" -out /home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage4.pncA.blastn.txt
```

### Analyze BLAST results

In [21]:
genes = ["gyrA", "pncA"]

In [23]:
for lin_i in range(1, 5):
    for gene in genes:
        df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage{}.{}.blastn.txt".format(lin_i, gene),
                         sep="\t",
                         names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
        display(df[["evalue", "bitscore", "length", "qseq"]])

,evalue,bitscore,length,qseq
0,0.0,4604,2517,ATGACAGACACGACGTTGCCGCCTGACGACTCGCTCGACCGGATCG...


,evalue,bitscore,length,qseq
0,0.0,1037,561,TCAGGAGCTGCAAACCAACTCGACGCTGGCGGTGCGCATCTCCTCC...


,evalue,bitscore,length,qseq
0,0.0,4626,2517,ATGACAGACACGACGTTGCCGCCTGACGACTCGCTCGACCGGATCG...


,evalue,bitscore,length,qseq
0,0.0,1037,561,TCAGGAGCTGCAAACCAACTCGACGCTGGCGGTGCGCATCTCCTCC...


,evalue,bitscore,length,qseq
0,0.0,4632,2517,ATGACAGACACGACGTTGCCGCCTGACGACTCGCTCGACCGGATCG...


,evalue,bitscore,length,qseq
0,0.0,1031,561,TCAGGAGCTGCAAACCAACTCGACGCTGGCGGTGCGCATCTCCTCC...


,evalue,bitscore,length,qseq
0,0.0,4632,2517,ATGACAGACACGACGTTGCCGCCTGACGACTCGCTCGACCGGATCG...


,evalue,bitscore,length,qseq
0,0.0,1037,561,TCAGGAGCTGCAAACCAACTCGACGCTGGCGGTGCGCATCTCCTCC...


In [25]:
# 7581 gyrA
# chrom_start and chrom_stop are inclusive and 1 indexed

mutate_ix = 7581

h37rv_ref_base = h37rv_seq[mutate_ix-5:mutate_ix+6]

chrom_start, chrom_stop = h37rv_summary.loc["gyrA", "chrom_start"], h37rv_summary.loc["gyrA", "chrom_stop"]

gene_ix = mutate_ix - chrom_start + 1

l1_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage1.gyrA.blastn.txt",
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l1_gyrA_seq = l1_df.qseq.values[0]
l2_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage2.gyrA.blastn.txt",
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l2_gyrA_seq = l2_df.qseq.values[0]
l3_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage3.gyrA.blastn.txt",
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l3_gyrA_seq = l3_df.qseq.values[0]
l4_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage4.gyrA.blastn.txt",
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l4_gyrA_seq = l4_df.qseq.values[0]

print("lineage 1 >>", mutate_ix, h37rv_ref_base, l1_gyrA_seq[gene_ix-5:gene_ix+6])
print("lineage 2 >>", mutate_ix, h37rv_ref_base, l2_gyrA_seq[gene_ix-5:gene_ix+6])
print("lineage 3 >>", mutate_ix, h37rv_ref_base, l3_gyrA_seq[gene_ix-5:gene_ix+6])
print("lineage 4 >>", mutate_ix, h37rv_ref_base, l4_gyrA_seq[gene_ix-5:gene_ix+6])


lineage 1 >> 7581 CTACGACAGCC CTACGACACCC
lineage 2 >> 7581 CTACGACAGCC CTACGACACCC
lineage 3 >> 7581 CTACGACAGCC CTACGACACCC
lineage 4 >> 7581 CTACGACAGCC CTACGACACCC


In [58]:
gyrA_seq[0:10], l1_gyrA_seq[0:10]

('ATGACAGACA', 'ATGACAGACA')

In [29]:
l1_df.mismatch, l2_df.mismatch, l3_df.mismatch, l4_df.mismatch

(0    8
 Name: mismatch, dtype: int64,
 0    4
 Name: mismatch, dtype: int64,
 0    3
 Name: mismatch, dtype: int64,
 0    3
 Name: mismatch, dtype: int64)

In [ ]:
l1_gyrA_seq == gyrA_seq, l2_gyrA_seq == gyrA_seq, l3_gyrA_seq == gyrA_seq, l4_gyrA_seq == gyrA_seq

In [49]:
gyrA_seq = h37rv_seq[h37rv_summary.loc["gyrA", "chrom_start"]-1:h37rv_summary.loc["gyrA", "chrom_stop"]]

for i in range(len(l1_gyrA_seq)):
    if gyrA_seq[i] != l1_gyrA_seq[i]:
        print(i, end=" ")
print()

60 283 669 1150 1745 1841 2002 2141 


In [33]:
# 2289049 pncA
# chrom_start and chrom_stop are inclusive and 1 indexed

mutate_ix = 2289049

h37rv_ref_base = h37rv_seq[mutate_ix-5:mutate_ix+6]

chrom_start, chrom_stop = h37rv_summary.loc["pncA", "chrom_start"], h37rv_summary.loc["pncA", "chrom_stop"]

gene_ix = mutate_ix - chrom_start + 1

l1_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage1.pncA.blastn.txt".format(lin_i, gene),
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l1_pncA_seq = l1_df.qseq.values[0]
l2_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage2.pncA.blastn.txt".format(lin_i, gene),
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l2_pncA_seq = l2_df.qseq.values[0]
l3_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage3.pncA.blastn.txt".format(lin_i, gene),
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l3_pncA_seq = l3_df.qseq.values[0]
l4_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage4.pncA.blastn.txt".format(lin_i, gene),
                 sep="\t",
                 names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
l4_pncA_seq = l4_df.qseq.values[0]

print("lineage 1 >>", mutate_ix, h37rv_ref_base, l1_pncA_seq[gene_ix-5:gene_ix+6])
print("lineage 2 >>", mutate_ix, h37rv_ref_base, l2_pncA_seq[gene_ix-5:gene_ix+6])
print("lineage 3 >>", mutate_ix, h37rv_ref_base, l3_pncA_seq[gene_ix-5:gene_ix+6])
print("lineage 4 >>", mutate_ix, h37rv_ref_base, l4_pncA_seq[gene_ix-5:gene_ix+6])


lineage 1 >> 2289049 GAGGAATAGTC GAGGAATAGTC
lineage 2 >> 2289049 GAGGAATAGTC GAGGAATAGTC
lineage 3 >> 2289049 GAGGAATAGTC GAAGAATAGTC
lineage 4 >> 2289049 GAGGAATAGTC GAGGAATAGTC


In [65]:
l3_df.mismatch

0    1
Name: mismatch, dtype: int64

In [64]:
gene_ix

369

In [62]:
pncA_seq = h37rv_seq[h37rv_summary.loc["pncA", "chrom_start"]-1:h37rv_summary.loc["pncA", "chrom_stop"]]

for i in range(len(l3_pncA_seq)):
    if pncA_seq[i] != l3_pncA_seq[i]:
        print(i, end=" ")
print()

366 


## Create list of mutate_ix for each lineage

In [34]:
DR_mut_kmers_df = pd.read_csv("./DR_mut_h37rv_kmers.csv")
DR_mut_kmers_df.head()

,h37rv_ix,mut_base,kmer
0,2288777,G,CCAGCACCCTG
1,2289071,G,GAGAAGTGGTCAC
2,2289049,G,GAGGAATAGTC
3,2288961,C,TTGTAGAACACCGCC
4,2289096,A,GGAAGTCCTTGGT


In [35]:
with open(l1_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L1_seq = line.strip()
    while line:
        line = in_f.readline()
        L1_seq += line.strip()
        
with open(l2_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L2_seq = line.strip()
    while line:
        line = in_f.readline()
        L2_seq += line.strip()
        
with open(l3_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L3_seq = line.strip()
    while line:
        line = in_f.readline()
        L3_seq += line.strip()

with open(l4_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L4_seq = line.strip()
    while line:
        line = in_f.readline()
        L4_seq += line.strip()
        
L_seqs = {1: L1_seq, 2: L2_seq, 3: L3_seq, 4: L4_seq}

In [36]:
problematic_ix = {1: [7581], 2: [7581], 3: [7581, 2289049], 4: [7581]}

In [37]:
problematic_ix_gene_map = {7581: "gyrA", 2289049: "pncA"}

In [38]:
c = pd.DataFrame(columns=["h37rv_ix", "L1_ix", "L2_ix", "L3_ix", "L4_ix", "ref_base", "mut_base"]).set_index("h37rv_ix")

for i in DR_mut_kmers_df.index:
    
    h37rv_ix = DR_mut_kmers_df.loc[i, "h37rv_ix"]
    h37rv_base = h37rv_seq[h37rv_ix]
    mut_base = DR_mut_kmers_df.loc[i, "mut_base"]
    
    kmer = DR_mut_kmers_df.loc[i, "kmer"]
    kmer_length = len(kmer)
    
    L_ix = {}
    for lin_i in range(1, 5):
        
        if h37rv_ix not in problematic_ix[lin_i]:
            match_start = L_seqs[lin_i].find(kmer)    
            L_ix[lin_i] = match_start + (kmer_length-1)//2

        else:
            gene = problematic_ix_gene_map[h37rv_ix]
            gene_h37rv_start = h37rv_summary.loc[gene, "chrom_start"]
            gene_ix = h37rv_ix - gene_h37rv_start + 1
            
            blast_df = pd.read_csv("/home/sm624/projects/mixed_calls/benchmarking/comprehensive/lineage123/blast/lineage{}.{}.blastn.txt".format(lin_i, gene),
                                sep="\t",
                                names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qseq", "nident"])
            
            L_start_ix = blast_df.loc[0, "qstart"] #1-indexed
            L_ix[lin_i] = L_start_ix + gene_ix - 1
    
    
        assert(h37rv_base == L_seqs[lin_i][L_ix[lin_i]])
        
    L1234_mut_ix_df.loc[h37rv_ix] = [L_ix[1], L_ix[2], L_ix[3], L_ix[4], h37rv_base, mut_base]
    
L1234_mut_ix_df

,L1_ix,L2_ix,L3_ix,L4_ix,ref_base,mut_base
h37rv_ix,,,,,,
2288777,2310818,2288812,2298581,2301252,A,G
2289071,2311112,2289106,2298875,2301546,T,G
2289049,2311090,2289084,2298853,2301524,A,G
2288961,2311002,2288996,2298765,2301436,A,C
2289096,2311137,2289131,2298900,2301571,C,A
2288919,2310960,2288954,2298723,2301394,C,A
7569,7569,8927,7569,7569,C,T
2288816,2310857,2288851,2298620,2301291,G,T
4247468,4257279,4257016,4257985,4252610,A,G


In [41]:
L1234_mut_ix_df.to_csv("./DR_mut_lineage_ix.csv")

# Modification

In [42]:
L123_mut_ix_df = pd.read_csv("./DR_mut_lineage_ix.csv")
L123_mut_ix_df.head()

,h37rv_ix,L1_ix,L2_ix,L3_ix,L4_ix,ref_base,mut_base
0,2288777,2310818,2288812,2298581,2301252,A,G
1,2289071,2311112,2289106,2298875,2301546,T,G
2,2289049,2311090,2289084,2298853,2301524,A,G
3,2288961,2311002,2288996,2298765,2301436,A,C
4,2289096,2311137,2289131,2298900,2301571,C,A


In [11]:
with open(l1_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L1_seq = line.strip()
    while line:
        line = in_f.readline()
        L1_seq += line.strip()
        
with open(l2_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L2_seq = line.strip()
    while line:
        line = in_f.readline()
        L2_seq += line.strip()
        
with open(l3_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L3_seq = line.strip()
    while line:
        line = in_f.readline()
        L3_seq += line.strip()
        
L1_seq_array = [x for x in L1_seq]
L2_seq_array = [x for x in L2_seq]
L3_seq_array = [x for x in L3_seq]

In [12]:
for i in L123_mut_ix_df.index:
    
    L1_ix = L123_mut_ix_df.loc[i, "L1_ix"]
    L2_ix = L123_mut_ix_df.loc[i, "L2_ix"]
    L3_ix = L123_mut_ix_df.loc[i, "L3_ix"]
    
    ref_base, mut_base = L123_mut_ix_df.loc[i, "ref_base"], L123_mut_ix_df.loc[i, "mut_base"]
    
    assert(L1_seq_array[L1_ix] == ref_base)
    L1_seq_array[L1_ix] = mut_base
    
    assert(L2_seq_array[L2_ix] == ref_base)
    L2_seq_array[L2_ix] = mut_base
    
    assert(L3_seq_array[L3_ix] == ref_base)
    L3_seq_array[L3_ix] = mut_base

In [13]:
out_dir = "./fastas_final"

In [23]:
L1_seq_main = "".join(L1_seq_array)
with open("{}/L1_mutant/L1_mutant.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L1-mutant\n")
    n_chunks = len(L1_seq_main)//80
    for i in range(n_chunks):
        out_f.write(L1_seq_main[i*80:i*80+80] + "\n")
    out_f.write(L1_seq_main[(i+1)*80:] + "\n")
    
L2_seq_main = "".join(L2_seq_array)
with open("{}/L2_mutant/L2_mutant.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L2-mutant\n")
    n_chunks = len(L2_seq_main)//80
    for i in range(n_chunks):
        out_f.write(L2_seq_main[i*80:i*80+80] + "\n")
    out_f.write(L2_seq_main[(i+1)*80:] + "\n")

L3_seq_main = "".join(L3_seq_array)
with open("{}/L3_mutant/L3_mutant.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L3-mutant\n")
    n_chunks = len(L3_seq_main)//80
    for i in range(n_chunks):
        out_f.write(L3_seq_main[i*80:i*80+80] + "\n")
    out_f.write(L3_seq_main[(i+1)*80:] + "\n")

In [24]:
with open("{}/L1_mutant/L1.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L1\n")
    n_chunks = len(L1_seq)//80
    for i in range(n_chunks):
        out_f.write(L1_seq[i*80:i*80+80] + "\n")
    out_f.write(L1_seq[(i+1)*80:] + "\n")
    
with open("{}/L2_mutant/L2.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L2\n")
    n_chunks = len(L2_seq)//80
    for i in range(n_chunks):
        out_f.write(L2_seq[i*80:i*80+80] + "\n")
    out_f.write(L2_seq[(i+1)*80:] + "\n")

with open("{}/L3_mutant/L3.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L3\n")
    n_chunks = len(L3_seq)//80
    for i in range(n_chunks):
        out_f.write(L3_seq[i*80:i*80+80] + "\n")
    out_f.write(L3_seq[(i+1)*80:] + "\n")

In [16]:
L123_mut_ix_df.head()

,h37rv_ix,L1_ix,L2_ix,L3_ix,ref_base,mut_base
0,2288777,2310818,2288812,2298581,A,G
1,2289071,2311112,2289106,2298875,T,G
2,2289049,2311090,2289084,2298853,A,G
3,2288961,2311002,2288996,2298765,A,C
4,2289096,2311137,2289131,2298900,C,A


In [22]:
L1_df = pd.DataFrame(columns=["POS", "REF", "ALT", "TYPE"])
L1_df["POS"] = L123_mut_ix_df.L1_ix+1
L1_df["REF"] = L123_mut_ix_df.ref_base
L1_df["ALT"] = L123_mut_ix_df.mut_base
L1_df["TYPE"] = ["DR"]*L123_mut_ix_df.shape[0]

L2_df = pd.DataFrame(columns=["POS", "REF", "ALT", "TYPE"])
L2_df["POS"] = L123_mut_ix_df.L2_ix+1
L2_df["REF"] = L123_mut_ix_df.ref_base
L2_df["ALT"] = L123_mut_ix_df.mut_base
L2_df["TYPE"] = ["DR"]*L123_mut_ix_df.shape[0]

L3_df = pd.DataFrame(columns=["POS", "REF", "ALT", "TYPE"])
L3_df["POS"] = L123_mut_ix_df.L3_ix+1
L3_df["REF"] = L123_mut_ix_df.ref_base
L3_df["ALT"] = L123_mut_ix_df.mut_base
L3_df["TYPE"] = ["DR"]*L123_mut_ix_df.shape[0]

L1_df.to_csv("./mutant_summaries_final/L1_mutant.csv", index=False)
L2_df.to_csv("./mutant_summaries_final/L2_mutant.csv", index=False)
L3_df.to_csv("./mutant_summaries_final/L3_mutant.csv", index=False)

In [3]:
L123_mut_ix_df = pd.read_csv("./DR_mut_lineage_ix.csv")
L123_mut_ix_df.head()

,h37rv_ix,L1_ix,L2_ix,L3_ix,ref_base,mut_base
0,2288777,2310818,2288812,2298581,A,G
1,2289071,2311112,2289106,2298875,T,G
2,2289049,2311090,2289084,2298853,A,G
3,2288961,2311002,2288996,2298765,A,C
4,2289096,2311137,2289131,2298900,C,A


In [4]:
L123_df = pd.DataFrame(columns=["POS", "REF", "ALT", "TYPE"])
L123_df["POS"] = L123_mut_ix_df.h37rv_ix+1
L123_df["REF"] = L123_mut_ix_df.ref_base
L123_df["ALT"] = L123_mut_ix_df.mut_base
L123_df["TYPE"] = ["DR"]*L123_mut_ix_df.shape[0]

L123_df.to_csv("./mutant_summaries_final/L123_mutant.csv", index=False)

## L4 addition

In [43]:
with open(l4_fa_path, "r") as in_f:
    header = in_f.readline()
    
    line = in_f.readline()
    L4_seq = line.strip()
    while line:
        line = in_f.readline()
        L4_seq += line.strip()
        
L4_seq_array = [x for x in L4_seq]

In [45]:
for i in L123_mut_ix_df.index:
    
    L4_ix = L123_mut_ix_df.loc[i, "L4_ix"]
    
    ref_base, mut_base = L123_mut_ix_df.loc[i, "ref_base"], L123_mut_ix_df.loc[i, "mut_base"]
    
    assert(L4_seq_array[L4_ix] == ref_base)
    L4_seq_array[L4_ix] = mut_base

In [46]:
out_dir = "./fastas_final"

L4_seq_main = "".join(L4_seq_array)
with open("{}/L4_mutant/L4_mutant.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L4-mutant\n")
    n_chunks = len(L4_seq_main)//80
    for i in range(n_chunks):
        out_f.write(L4_seq_main[i*80:i*80+80] + "\n")
    out_f.write(L4_seq_main[(i+1)*80:] + "\n")
    
with open("{}/L4_mutant/L4.fasta".format(out_dir), "w") as out_f:
    out_f.write(">L4\n")
    n_chunks = len(L4_seq)//80
    for i in range(n_chunks):
        out_f.write(L4_seq[i*80:i*80+80] + "\n")
    out_f.write(L4_seq[(i+1)*80:] + "\n")

In [47]:
L4_df = pd.DataFrame(columns=["POS", "REF", "ALT", "TYPE"])
L4_df["POS"] = L123_mut_ix_df.L4_ix+1
L4_df["REF"] = L123_mut_ix_df.ref_base
L4_df["ALT"] = L123_mut_ix_df.mut_base
L4_df["TYPE"] = ["DR"]*L123_mut_ix_df.shape[0]

L4_df.to_csv("./mutant_summaries_final/L4_mutant.csv", index=False)